## Description

This Jupyter notebook is part of the "Time History Buckling Analysis of EMULF Delta Floater" tutorial. The user is assumed to have exported a capacity model from GeniE, after which a few steps are necessary in HydroD and Sima/Bladed (Sima is assumed in this tutorial). Those steps are not part of this tutorial and can be skipped as the resulting input files are provided. 

In this notebook Python is used with OneWorkflow to set up the analysis which includes running Wasim to construct panel loads based on forces and motions previously computed in Sima. Subsequently, Sestra and SesamCore compute the structural response and verifies the capacity model against buckling according to DNV-RP-C201.

The capacity model comes as a single JSON file from GeniE, control data is read from a spreadsheet into a dictionary (Python), including file names for coupled analysis result, as well as start and stop times for Wasim load reconstruction and load transfer. 

<img src="EMULF_Deltafloater.png" alt="Drawing" style="width: 700px;"/>

Results are currently stored in one .csv and one .lis file for each stiffened plate group.
Graphics of reaction force in z-direction from Sestra are presented for each design load case (DLC).
Buckling results are presented as a table of worst usage factor over all time steps and load cases for each stiffened plate group. Note that the first two time steps are ignored, to avoid unrealistic result peaks.

The workflow is split into several cells which quickly summarize the procedure:

- Set up files and folders for the workflow
- Read DLCs from a spreadsheet
- Create tasks for each DLC
- Run the analysis
- Print Z reaction force per DLC
- Buckling results agreggation over all DLCs



## Set up files and folders for the workflow

Some general workflow properties are setup and the OneWorkflow client is created. The client will be important to manage and execute the workflow.

In [ ]:
from pathlib import Path
import os, sys
from dnv.oneworkflow.utils import *


####### USER INPUT #########

sys.path.append("../../PythonModules")      # Relative path to PythonModules
workspaceId = "FOWT_ULS"                    # Set the Id for the workspace
cloudRun = False                            # Set whether to run in the cloud ( in this example set to False)

#### END of USER INPUT ####


root_folder = os.getcwd()                                   #The root folder is set as the path to the current working directory of the notebook. 
workspacePath = str(Path(root_folder, 'Workspace'))         #The workspace path is set to a subfolder "Workspace" inside the root folder

workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, inplace_execution = True,auto_deploy_option= AutoDeployOption.TEST)


Related to the folders defined above, it is assumed in the code that common input files for the workflow are gathered in a folder "CommonFiles" inside the workspace path. If you have not yet, please copy the folder "CommonFiles" from the input files of this tutorial inside the "Workspace" folder.

## Read DLCs from a spreadsheet

In the code below the name of the spreadsheet is declared as well as the superelement type for the structural model and the prefix. It will be assumed that the structural model file is named <"model_prefix">T<"topsuper">.FEM.
In the parameter mapping an association of parameters is done which will be used for assigning each entry in the spreadsheet a corresponding variable. 

In [ ]:
import pandas as pd

####### USER INPUT #########

dlc_sheet = "DLC_input.xlsx"    # The name of the spreadsheet with the parameters of each DLC
topsuper = 2                    # superelement type number for the structural model
model_prefix ="FOWT_ULS_"       # prefix for the structural model
capacity = "COL2_ULS.json"      # Name of the capacity model

#### END of USER INPUT ####

# In the definition below the parameter mapping is defined which associates each column in the excel sheet with a corresponding variable in the code. If you use an excel sheet with a different structure, this is where the workflow can be adjusted to interpret that sheet:

parameter_mapping = {'Group': "group",
				'LoadCase': "loadcase",
				"SimaFilesFolder": "sima_files_folder",
				"SimaFilesName": "sima_files_name",
                "TimeStep": "timestep",
				"Depth": "depth",
				"StartWasimSolve": "start_solve",
				"StopWasimSolve": "stop_solve",
            	"StartWasimStru": "start_stru",
				"StopWasimStru": "stop_stru",
				"NSteps": "nsteps",
				"StartSestra": "start_sestra",
				"StopSestra": "stop_sestra",
				"StartBuckling": "BUCKLINGSTART",
				"StopBuckling": "BUCKLINGEND",
				                }

input_data = pd.read_excel(os.path.join(workspacePath, dlc_sheet), index_col=1) # in here the excel sheet is read


fixed_parameters_all_load_cases = {
    'mor_sel': topsuper,                                             # superelement number for the morison model, in this case the same as the structural model
    'prefix': model_prefix,                                          # prefix for the morison (and structural) model
    'cap_mod': capacity,                                             # name of the capacity model, for the input.json
    'modelf' : model_prefix+"T"+str(topsuper) + ".FEM",          # name of the model file, for the input.json
    'loadf' :  model_prefix+"L"+str(topsuper) + ".FEM"           # name of the load file, for the input.json 
    }


## Create tasks for each DLC

In the code below the tasks are created for each analysis and for each DLC. The user may select to skip part of the workflow in the USER INPUT section, in that case make sure all necessary files for the following analysis are available. For example, if Wasim has already been run, the user might opt to skip the Wasim run, but must ensure that the load files remain in the DLC folders. Moreover, to avoid deleting these files at the begining of the run then declare `clean = False`.

- The tasks for all Wasim steps are easily created with the help of `WasimTaskCreator`. 
- The template files are populated with the proper values for each DLC using the [`CreateInputFileFromFileTemplateCommand`](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.oneworkflow.html#dnv.oneworkflow.create_input_file_from_file_template_command.CreateInputFileFromFileTemplateCommand)

In [ ]:
import os
import shutil
from WasimTaskCreator import *
from dnv.sesam.commands import *
from dnv.oneworkflow import *

####### USER INPUT #########

# Set True for the workflow steps to be run, and False for the steps to be skipped:

run_wasim = True                          # Define whether to run Wasim (setup, solve, snapshots and stru)
run_sestra_template_command = True        # Define whether to update sestra input files with the latest variables
run_sesam_core_template_command = True    # Define whether to update sesam core input files with the latest variables
run_sestra_sesam_core = True              # Define whether to run sestra and sesam core
run_sesam_core_input_command = True

clean = True                              # Define whether to clean (True) or keep (False) files (input and ouptut) from a previous run, if skipping any anlysis step this should be set to false.


#### END of USER INPUT ####

if clean:
    shutil.rmtree(os.path.join(workspacePath,"LoadCases"), ignore_errors=True)

try:                                      # Check whether the LoadCases folder exists
    os.chdir(os.path.join(workspacePath,"LoadCases"))
except:
    print("LoadCases folder not found")

workflow_sequence = []
tasks = []

display(input_data)

# In the below a loop over all DLCs is run to get the parameters for each DLC, creating the DLC folders and setup the tasks.

for casename, case in input_data.iterrows():                
    casedict = case.to_dict()
    input_parameters = {}
    
    for key, value in case.items():
        input_parameters[parameter_mapping[key]] = str(value)

    get_folder=os.path.join(workspacePath,input_parameters["sima_files_folder"])                               # Define the  path to the folder where the Sima files for this DLC are stored
    DLC_folder=os.path.join(workspacePath,"LoadCases",case.name)                                               # Define the path of the DLC folder, where the analysis will be run
    isExist = os.path.exists(DLC_folder)
    if not isExist:          
        os.makedirs(DLC_folder)                                                                                # Create the DLC folder, if it does not exist already
    
    shutil.copytree(os.path.join(get_folder),os.path.join(DLC_folder),dirs_exist_ok=True)                      # Copy files from the Sima files folder to the DLC folder

    input_parameters = input_parameters | fixed_parameters_all_load_cases
    print("The following parameters are used for load case: " + casename)
    print(json.dumps(input_parameters, indent=4, sort_keys=True))

    #Wasimtaskcreator is a function that simplifies the creation of wasim tasks, it helps setup separate calm sea and dynamic runs in Wasim so that the load factors can be applied later in the analysis
    
    if run_wasim:
        tasks = WasimTaskCreator().CreateTasks(input_parameters)
        
    sestra_template_command = CreateInputFileFromFileTemplateCommand(                                       # Create the sestra.inp by taking the sestra_template.inp and replacing the strings with parameters for the DLC
        template_input_file  = "sestra_template.inp",
        input_filename  = "sestra.inp",
        parameters= input_parameters
    )

    if run_sestra_template_command:
        tasks.append(sestra_template_command)    
 
    sesam_core_template_command = CreateInputFileFromFileTemplateCommand(                                   # Create the SesamCore_buckling.jnl by taking the SesamCore_buckling_template.jnl and replacing the strings with parameters for the DLC
        template_input_file  = "SesamCore_buckling_template.jnl",
        input_filename  = "SesamCore_buckling.jnl",
        parameters= input_parameters
    )
    if run_sesam_core_template_command:
        tasks.append(sesam_core_template_command)
    
    sesam_core_input_command = CreateInputFileFromFileTemplateCommand(                                      # Create the SesamCore_buckling.jnl by taking the SesamCore_buckling_template.jnl and replacing the strings with parameters for the DLC
        template_input_file  = "input_template.json",
        input_filename  = "input.json",
        parameters= input_parameters
    )

    if run_sesam_core_input_command:                                                                       # Create the input.json by taking the inpu_template.json and replacing the strings with parameters for the DLC
        tasks.append(sesam_core_input_command)


    if run_sestra_sesam_core:    
        tasks.append(SesamCoreCommand(command = "uls",input_file_name= "input.json", options = "-v"))
    
    workflow_sequence.append(CommandInfo(commands=tasks,load_case_foldername=casename))                       # Add the generated tasks to the workflow sequence.

## Run the analysis

In the code below we run the analysis using [`run_managed_commands_in_parallel_async`](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.oneworkflow.utils.html#dnv.oneworkflow.utils.workflow_commands_runners_parallel.run_managed_commands_in_parallel_async) with the previously created workflow client and the commands setup in the previous cell. We set `log_job=false` to get a more restricted log, but it can be set to `True` for more detailed logs if wanted. We set `enable_common_files_copy_to_load_cases=True` to copy files from the "CommonFiles" folder into every single DLC folder. 

Subsequently, the `analysisStatusChecker` is used to check whether the analysis was successful by reading Wasim and Sesam Core output files. 

In [ ]:
## Run the analysis

print("Running commands in parallel")
await run_managed_commands_in_parallel_async(
             client=workflow_client,
             commands_info=workflow_sequence,
             log_job=False,
             enable_common_files_copy_to_load_cases=True,
 )


## Check if analysis is successful 

local__result_path = Path(workspacePath, workflow_client.results_directory)
os.chdir(local__result_path)

print("Verifying Wasim results:")
wasim_succeeded_text = "FINISHED: SUCCESS"
wasim_files_to_check = {"Wasim solve":'wasim_solve.lis', "Wasim stru" : 'wasim_stru.lis'}

import analysisStatusChecker
analysisStatusChecker.checkStatus(wasim_files_to_check,wasim_succeeded_text)


print("Verifying Sesam Core results:")
score_succeeded_text = "Duration:"
score_to_check = {"Sesam Core":'SCORE.MLG'}
analysisStatusChecker.checkStatus(score_to_check,score_succeeded_text)


## Print Z reaction forces per DLC

In [ ]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")

print("Current Time =", current_time)
print("Current Date =", current_date)

def format_time(value, _):
    return "{:.1e}".format(value)  # Format the time with two decimal places
for loadcase_folder_name, _ in input_data.iterrows():   
    result_path = os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name,model_prefix+"_reactions_history1.csv")
    reaction_history = pd.read_csv(result_path)
    df = pd.read_csv(result_path, delimiter=';') 
    from matplotlib import pylab as plt
    from matplotlib.ticker import FuncFormatter
    # Set the default font size for all labels

    time = df['Time']
    fx = df['FX']
    fy = df['FY']
    fz = df['FZ']
    mx = df['MX']
    my = df['MY']
    mz = df['MZ']

    # Define a custom formatting function for x-axis values
   

    # Plot a 2D graph
    plt.plot(time, fz, marker='.', label='FZ')

    # Add labels and title
    plt.xlabel('Result case id', fontsize=10)
    plt.ylabel('Force', fontsize=10)
    plt.xticks(fontsize=10)
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_time))
    plt.yticks(fontsize=10)
    plt.title('Reaction z-force over time-history', fontsize=14)
    plt.legend(fontsize=10)  # Show legend with labels

    # Add gridlines
    plt.grid(True, linestyle='--', alpha=0.7)

    # Set x-axis limits
    plt.xlim(min(time), max(time))


    # Show the plot
    plt.show()
    plt.savefig(os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name,"reaction_forces_plot.png"))

## Buckling results aggregation over all DLCs
Print maximum usage factor and criterion for worst stiffener on each plate group, over all load cases

In [ ]:
import os
from pathlib import Path
import glob
import pandas as pd

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")

print("Current Time =", current_time)
print("Current Date =", current_date)

panels_and_cases = pd.DataFrame()
panel_names = set()
for loadcase_folder_name, _ in input_data.iterrows():
    result_folder = os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name)
        
    os.chdir(result_folder)

    # loop over all panel result files (csv)
    # Set threshold value for removing the first few result cases to omit extreme initial condition results

    schema={
        "math score": int
    }
    
    for file in glob.glob('SesamCore_buckling_*Panel*.csv'):
        gen = pd.read_csv(file, dtype=schema, chunksize=10000000)
        panel = pd.concat((x.query("`Result case id` >= 3") for x in gen), ignore_index=True)
        panel['Plate field'] = Path(file).stem
        panel['LoadCase'] = loadcase_folder_name
        if not panel.empty:
            panels_and_cases = pd.concat([panels_and_cases,panel])
            panel_names.add(Path(file).stem)
        
df2 = pd.DataFrame(columns=['Stiffener name','Plate field', 'UfMax','UfMax criterion','LoadCase', 'Time'])
for panel_name in panel_names:
    df = panels_and_cases.query('`Plate field` == @panel_name')
    df.reset_index(inplace=True)
    if not df.empty:
        df2.loc[len(df2.index)] = dict(df.iloc[df['UfMax'].idxmax()])
df2.sort_values(by='UfMax',ascending=False,inplace=True)
print(f"Maximum Uf for each panel, taken over all load cases and result cases.")
print(f"Note that the first two time steps (usually extreme results) have been filtered out to avoid reporting unrealistic peak results.")
display(df2)
